In [2]:
# Train of mnsist cnn

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Input
from keras import backend as K

(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, 28, 28)
    x_test = x_test.reshape(x_test.shape[0], 1, 28, 28)
    input_shape = (1, 28, 28)
else:
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
    x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
    input_shape = (28, 28, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

model = Sequential()
model.add(Input(shape=input_shape))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))


model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])



Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 52s 110ms/step - accuracy: 0.8474 - loss: 0.4842 - val_accuracy: 0.9840 - val_loss: 0.0511
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 55s 117ms/step - accuracy: 0.9734 - loss: 0.0923 - val_accuracy: 0.9875 - val_loss: 0.0371
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 52s 110ms/step - accuracy: 0.9820 - loss: 0.0619 - val_accuracy: 0.9901 - val_loss: 0.0313
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 52s 110ms/step - accuracy: 0.9841 - loss: 0.0526 - val_accuracy: 0.9894 - val_loss: 0.0330
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 52s 111ms/step - accuracy: 0.9857 - loss: 0.0476 - val_accuracy: 0.9911 - val_loss: 0.0313
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 55s 117ms/step - accuracy: 0.9877 - loss: 0.0395 - val_accuracy: 0.9909 - val_loss: 0.0302
Epoch 7/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 56s 119ms/step - accuracy: 0.9885 - loss: 0.0358 - val_accuracy: 0.9914 - val_loss: 0.0273
Epoch 8/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 50s 106ms/step - accuracy: 0.9888 - loss: 0

In [ ]:
model.save('mnist_cnn_model.keras')

In [1]:
#train of species cnn

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2
import json

# Pfad zu Ihrem Hauptordner
base_dir = '/Users/MeinNotebook/Google Drive/Meine Ablage/Scans/notebooks/Daten - DH/Vogelarten'

# Bildgrößen und Batchgröße definieren
img_height, img_width = 22, 150
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2)

# Keine Datenanreicherung für Validierungsdaten
validation_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

validation_generator = validation_datagen.flow_from_directory(
    base_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

# CNN Modell erstellen

model = Sequential([
    Input(shape=(img_height, img_width, 3)),
    Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
    MaxPooling2D(2, 2),
    Dropout(0.1),
    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
    MaxPooling2D(2, 2),
    Dropout(0.1),
    Flatten(),
    Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
    Dense(len(train_generator.class_indices), activation='softmax')
])


class_indices = train_generator.class_indices
with open('/Users/MeinNotebook/Google Drive/Meine Ablage/Scans/class_indices.json', 'w') as file:
    json.dump(class_indices, file)
    
    
    
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks definieren
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
model_checkpoint = ModelCheckpoint('vogelarten_best_model.keras', save_best_only=True)

# Modell trainieren
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=50,  # Anzahl der Epochen kann angepasst werden
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[early_stopping, model_checkpoint]
)

Found 2240 images belonging to 28 classes.
Found 560 images belonging to 28 classes.


/Users/MeinNotebook/DocDig/.venv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 2300s 33s/step - accuracy: 0.0376 - loss: 4.4970 - val_accuracy: 0.0404 - val_loss: 3.6300
Epoch 2/50


2024-10-05 23:52:13.397522: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/opt/homebrew/Cellar/python@3.12/3.12.6/Frameworks/Python.framework/Versions/3.12/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


70/70 ━━━━━━━━━━━━━━━━━━━━ 13s 181ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 3.6318
Epoch 3/50
 3/70 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.0260 - loss: 3.6265

...

70/70 ━━━━━━━━━━━━━━━━━━━━ 4s 54ms/step - accuracy: 0.0681 - loss: 3.3891 - val_accuracy: 0.1360 - val_loss: 3.2842
Epoch 6/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 545us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.1250 - val_loss: 3.2791
Epoch 7/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 4s 54ms/step - accuracy: 0.1362 - loss: 3.1849 - val_accuracy: 0.1857 - val_loss: 2.9936
Epoch 8/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 237us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.1250 - val_loss: 3.0554
Epoch 9/50
 3/70 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - accuracy: 0.1753 - loss: 2.9894

2024-10-05 23:52:37.941434: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


70/70 ━━━━━━━━━━━━━━━━━━━━ 4s 54ms/step - accuracy: 0.1855 - loss: 2.9487 - val_accuracy: 0.2188 - val_loss: 2.8151
Epoch 10/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 583us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.3125 - val_loss: 2.5590
Epoch 11/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 4s 53ms/step - accuracy: 0.2170 - loss: 2.8116 - val_accuracy: 0.2316 - val_loss: 2.7029
Epoch 12/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 276us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.2500 - val_loss: 2.5827
Epoch 13/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 4s 53ms/step - accuracy: 0.2510 - loss: 2.6777 - val_accuracy: 0.2482 - val_loss: 2.6629
Epoch 14/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 264us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.2500 - val_loss: 2.5759
Epoch 15/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 4s 57ms/step - accuracy: 0.2286 - loss: 2.6811 - val_accuracy: 0.2629 - val_loss: 2.6347
Epoch 16/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 250us/step - accuracy: 0.0000e+00 - loss: 0.000

2024-10-05 23:52:53.305368: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


70/70 ━━━━━━━━━━━━━━━━━━━━ 4s 54ms/step - accuracy: 0.2536 - loss: 2.5763 - val_accuracy: 0.2629 - val_loss: 2.5854
Epoch 18/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 656us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.4375 - val_loss: 1.9067
Epoch 19/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 4s 56ms/step - accuracy: 0.2723 - loss: 2.5226 - val_accuracy: 0.2757 - val_loss: 2.6154
Epoch 20/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 261us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.3750 - val_loss: 2.4124
Epoch 21/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 4s 55ms/step - accuracy: 0.2866 - loss: 2.5219 - val_accuracy: 0.2610 - val_loss: 2.6031
Epoch 22/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 271us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.2500 - val_loss: 2.8134
Epoch 23/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 4s 57ms/step - accuracy: 0.2846 - loss: 2.4901 - val_accuracy: 0.2555 - val_loss: 2.6669
Epoch 24/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 276us/step - accuracy: 0.0000e+00 - loss: 0.000

In [2]:
model.save('vogelarten_final_model.keras')